In [ ]:
#imports
from collections import Counter  
import pandas as pd
import numpy as np
from collections import defaultdict
#plotting tools
import seaborn as sn
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from copy import deepcopy

#stats module
from scipy import stats

#MOL2VEC related imports
import re, gc
from gensim.models import Word2Vec 
from mol2vec.features import mol2alt_sentence, mol2sentence, MolSentence, DfVec, sentences2vec
from gensim.models import word2vec

#rdkit
from rdkit import Chem
from rdkit.Chem import MACCSkeys

#ML models and splits from sklearn
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix, roc_auc_score, precision_recall_fscore_support,  accuracy_score

In [ ]:
#The default xlrd engine has dropped support for xlsx files
#REF.:https://stackoverflow.com/questions/65254535/xlrd-biffh-xlrderror-excel-xlsx-file-not-supported
xls = pd.ExcelFile('dataset_rdkit.xlsx', engine = 'openpyxl')
df1 = pd.read_excel(xls, 'passb1_commontargets')
df2 = pd.read_excel(xls, 'passb2_commontargets')
df3 = pd.read_excel(xls, 'fail_common_targets')
df = pd.concat([df1,df2, df3])
print(len(df))

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem


#Define clustering setup
def ClusterFps(fps,cutoff=0.2):
    from rdkit import DataStructs
    from rdkit.ML.Cluster import Butina

    # first generate the distance matrix:
    dists = []
    nfps = len(fps)
    for i in range(1,nfps):
        sims = DataStructs.BulkTanimotoSimilarity(fps[i],fps[:i])
        dists.extend([1-x for x in sims])

    # now cluster the data:
    cs = Butina.ClusterData(dists,nfps,cutoff,isDistData=True)
    return cs

In [ ]:
## Helper functions 
smi2vec_model = Word2Vec.load('word2vec.model')

#wrong ones dict is a dict used to correct a couple of SMI strings.
wrong_ones_dict = {
    '[H][N]([H])([H])[Pt](Cl)(Cl)[N]([H])([H])[H]' : 'C1CN2C(=NN=C2C(F)(F)F)CN1C(=O)CC(CC3=CC(=C(C=C3F)F)F)N',
    '[H][N]([H])([H])[Pt]1(OC(=O)C2(CCC2)C(=O)O1)[N]([H])([H])[H]': 'C1CC(C1)(C(=O)O)C(=O)O.[NH2-].[NH2-].[Pt+2]',
    '[H][N]1([H])[C@@H]2CCCC[C@H]2[N]([H])([H])[Pt]11OC(=O)C(=O)O1' : 'C1CCC(C(C1)[NH-])[NH-].C(=O)(C(=O)[O-])[O-].[Pt+4]'
}
def smi2mol(x):
    if x in wrong_ones_dict:
        return Chem.MolFromSmiles(wrong_ones_dict[x])
    return Chem.MolFromSmiles(x)

def smiles2vec(x):
    lst = []
    for word in re.findall(r'.{3}',x):
        if word in smi2vec_model.wv:
            lst.append(smi2vec_model.wv[word])
    return sum(lst)/len(lst)

def get_maccs_from_mol(mol):
    return np.array(MACCSkeys.GenMACCSKeys(mol))

def read_sheets(sheet_list):
    sheet_list = [pd.read_excel(xls, x) for x in sheet_list]
    return pd.concat(sheet_list)

def atc_splits(df):
    in_row = defaultdict(list)
    for idx, row in df.iterrows():
        for jj in intervention_atc[row['nct_id']]:
            in_row[jj].append(idx)

    freq_list = list(in_row.items())
    freq_list.sort(reverse = True, key = lambda x : len(x[1]))
    return freq_list

def get_hyperparams(X, y):
    classifiers1 = [
        GridSearchCV(RandomForestClassifier(n_jobs = -1), {'n_estimators':[100, 200,300,250],'criterion':["gini", "entropy"],'max_depth':[10,15,20, None]}, n_jobs  =-1, verbose = 3),
        GridSearchCV(ExtraTreesClassifier(n_jobs = -1),{'n_estimators':[100, 200,300,250],'criterion':["gini", "entropy"],'max_depth':[10,15,20, None]}, n_jobs  =-1, verbose = 3),
        GridSearchCV(AdaBoostClassifier(),{'n_estimators':[100, 200,300,250]}, n_jobs  =-1, verbose = 3),
        GridSearchCV(SVC(), {'kernel':['rbf'], 'gamma':["scale"] }, n_jobs  =-1, verbose = 3),
        GridSearchCV(LogisticRegression(), {'penalty' : ['l2', 'none']}, n_jobs  =-1),
        GridSearchCV(KNeighborsClassifier(), {'n_neighbors': list(range(10)), 'weights':["distance", "uniform"]}, n_jobs  =-1, verbose = 3),
        GridSearchCV(GaussianNB(), {'var_smoothing' : [1e-9]}, n_jobs  =-1, verbose = 3), 
        GridSearchCV(MLPClassifier(), {'hidden_layer_sizes' : [[100], [100, 200], [100, 150, 200]], "learning_rate_init":[0.01, 0.001, 0.05]}, n_jobs  =-1, verbose = 3)
    ]
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    classifiers1 = [x.fit(X_scaled, y).best_params_ for x in classifiers1]
    return classifiers1

def get_hyperparams_all(X, y):
    classifiers1 = [
        GridSearchCV(RandomForestClassifier(n_jobs = -1), {'n_estimators':[100, 200,300,250],'criterion':["gini", "entropy"],'max_depth':[10,15,20, None]}, n_jobs  =-1, verbose = 3),
        GridSearchCV(ExtraTreesClassifier(n_jobs = -1),{'n_estimators':[100, 200,300,250],'criterion':["gini", "entropy"],'max_depth':[10,15,20, None]}, n_jobs  =-1, verbose = 3),
        GridSearchCV(AdaBoostClassifier(),{'n_estimators':[100, 200,300,250]}, n_jobs  =-1, verbose = 3),
        GridSearchCV(SVC(), {'kernel':['rbf'], 'gamma':["scale"] }, n_jobs  =-1, verbose = 3),
        GridSearchCV(LogisticRegression(), {'penalty' : ['l2', 'none']}, n_jobs  =-1),
        GridSearchCV(KNeighborsClassifier(), {'n_neighbors': list(range(20)), 'weights':["distance", "uniform"]}, n_jobs  =-1, verbose = 3),
        GridSearchCV(GaussianNB(), {'var_smoothing' : [1e-9]}, n_jobs  =-1, verbose = 3), 
        GridSearchCV(MLPClassifier(), {'hidden_layer_sizes' : [[100], [100, 200], [100, 150, 200]], "learning_rate_init":[0.01, 0.001, 0.05]}, n_jobs  =-1, verbose = 3)
    ]
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    classifiers1 = [x.fit(X_scaled, y) for x in classifiers1]
    return classifiers1

def get_classifiers(classifiers1):
    classifiers = [
            #{0:1, 1:5}
            RandomForestClassifier(n_jobs = -1, **classifiers1[0],class_weight="balanced"),
            ExtraTreesClassifier(n_jobs = -1, **classifiers1[1],class_weight="balanced"),
            AdaBoostClassifier(**classifiers1[2]),
            SVC(**classifiers1[3],class_weight="balanced"),
            LogisticRegression(**classifiers1[4],class_weight="balanced"),
            KNeighborsClassifier(**classifiers1[5]),
            GaussianNB(**classifiers1[6]),
            MLPClassifier(**classifiers1[7])
        ]
    return classifiers

def get_classifiers_weighted(classifiers1):
    classifiers = [
            #{0:1, 1:5}
            RandomForestClassifier(n_jobs = -1, **classifiers1[0],class_weight={1:5, 0:1}),
            ExtraTreesClassifier(n_jobs = -1, **classifiers1[1],class_weight={1:5, 0:1}),
            AdaBoostClassifier(**classifiers1[2]),
            SVC(**classifiers1[3],class_weight={1:5, 0:1}),
            LogisticRegression(**classifiers1[4],class_weight={1:5, 0:1}),
            KNeighborsClassifier(**classifiers1[5]),
            GaussianNB(**classifiers1[6]),
            MLPClassifier(**classifiers1[7])
        ]
    return classifiers

def get_basic_classifiers():
    classifiers = [
            RandomForestClassifier(n_jobs = -1),
            ExtraTreesClassifier(n_jobs = -1),
            AdaBoostClassifier(),
            SVC(),
            LogisticRegression(),
            KNeighborsClassifier(),
            GaussianNB(),
            MLPClassifier()
        ]
    return classifiers

def get_train_test_split(X, y, split_indices):
    X1_test_set = []
    X1_test_set.extend(split_indices[1])
    X1_test_mask = np.zeros((X.shape[0],), bool)
    X1_test_mask[X1_test_set] = True
    X1_test = X[X1_test_mask]
    X1 = X[~X1_test_mask]
    y1 = y[~X1_test_mask]
    y1_test = y[X1_test_mask]
    
    return X1, X1_test, y1, y1_test

In [ ]:
# Invervention name : ATC group mapping
intervention_smi = defaultdict(list)
for idx, row in df.iterrows():
        intervention_smi[row['nct_id']].append(row['smi'])

cnt = 0
for intervention in intervention_smi:
    smi = set(intervention_smi[intervention])
    if len(smi) > 1:
        cnt+=1
assert cnt == 0, "Intervention with multiple smi strings!!!"

intervention_smi = list(intervention_smi.items())
intervention_smi = [(x[0], x[1][0]) for x in intervention_smi]
print(intervention_smi[0])
fps = [AllChem.GetMorganFingerprintAsBitVect(smi2mol(x[1]),2,1024) for x in intervention_smi]
clusters=ClusterFps(fps,cutoff=0.6)

In [ ]:
np.array(AllChem.GetMorganFingerprintAsBitVect(smi2mol(intervention_smi[0][1]),2,1024))

In [ ]:
lengths = []
for cutoff in list(np.arange(0.1, 1.5, 0.1)):
    clusters=ClusterFps(fps,cutoff=cutoff)
    lengths.append(len(clusters))

ax = sn.lineplot(x = list(np.arange(0.1, 1.5, 0.1)), y = lengths)
ax.set(xlabel='cutoff', ylabel='number of clusters')

In [ ]:
clusters = ClusterFps(fps, cutoff = 0.9)
print(f"The number of clusters: {len(clusters)}")

In [ ]:
print([len(i) for i in clusters])

In [ ]:
# Fingerprint functions
#accept a dataframe with the 'smi' field, and returns a numpy array of fingerprints for each row of the dataframe.
def mol2vec(df):
    df = df.copy(deep = True)
    df['mol'] = df['smi'].apply(smi2mol)
    model = word2vec.Word2Vec.load('../model_300dim.pkl')
    #Constructing sentences
    df['sentence'] = df.apply(lambda x: MolSentence(mol2alt_sentence(x['mol'], 1)), axis=1)
    #Extracting embeddings to a numpy.array
    #Note that we always should mark unseen='UNK' in sentence2vec() so that model is taught how to handle unknown substructures
    df['mol2vec'] = [DfVec(x) for x in sentences2vec(df['sentence'], model, unseen='UNK')]
    X_mol2vec = np.array([x.vec for x in df['mol2vec']])
    return X_mol2vec

def morgan(df):
    df = df.copy(deep = True)
    X_mol2vec = np.array([AllChem.GetMorganFingerprintAsBitVect(smi2mol(x),2,1024) for x in df['smi']])
    return X_mol2vec

def mol2vec_ft(df):
    df = df.copy(deep = True)
    df['mol'] = df['smi'].apply(smi2mol)
    model = word2vec.Word2Vec.load('../updatedmodel.pkl')
    #Constructing sentences
    df['sentence'] = df.apply(lambda x: MolSentence(mol2alt_sentence(x['mol'], 1)), axis=1)

    #Extracting embeddings to a numpy.array
    #Note that we always should mark unseen='UNK' in sentence2vec() so that model is taught how to handle unknown substructures
    df['mol2vec'] = [DfVec(x) for x in sentences2vec(df['sentence'], model, unseen='UNK')]
    X_mol2vec = np.array([x.vec for x in df['mol2vec']])
    return X_mol2vec

def smi2vec(df):
    df = df.copy(deep = True)
    df['smi2vec'] = df["smi"].apply(lambda x : smiles2vec(x))
    X_smi2vec = np.array([x for x in df['smi2vec']])
    return X_smi2vec
def maccs(df):
    df = df.copy(deep = True)
    df['mol'] = df['smi'].apply(smi2mol)
    maccs_keys = df['mol'].apply(get_maccs_from_mol)
    maccs_keys = maccs_keys.to_numpy()
    maccs_keys = np.asarray(list(maccs_keys))
    return maccs_keys
    
## Testing code for all the fingerpring modules
for fp in [mol2vec, mol2vec_ft, smi2vec, maccs, morgan]:
    x = fp(df)
    print(fp.__name__, x.shape)

In [ ]:
#various sheets of the dataset, for various experiments

sheets = {
    'main' : ['passb1_commontargets', 'passb2_commontargets', 'fail_common_targets', "p2fail_commontargets"],
    'ki' : ['passb1_common_ki', 'passb2_common_ki', 'fail_common_ki', "p2fail_common_ki"],
    'stdval' : ['passb1_common_stdval', 'passb2_common_stdval', 'fail_common_stdval', "p2fail_common_stdval"],
    'main_v1' : ['passb1_commontargets', 'passb2_commontargets', 'fail_common_targets'],
    'ki_v1' : ['passb1_common_ki', 'passb2_common_ki', 'fail_common_ki'],
    'stdval_v1' : ['passb1_common_stdval', 'passb2_common_stdval', 'fail_common_stdval'],
}

#fingerprints to be appended to the datasets using these functions
fingerprints = {
    'mol2vec': mol2vec,
    'mol2vec_ft' : mol2vec_ft,
    'smi2vec' : smi2vec,
    'maccs' : maccs
}

#base features to be included in all experiments
# base_features = ["label","smi",'mol_w', 'num_valence_electrons', 'num_heteroatoms', 'gastiger_charges', 'tpsa', 'h_acceptors',\
#             'h_donors', 'n_rotatable_bonds', 'n_rings', 'num_of_atoms', 'num_of_heavy_atoms', "count_drug_targets", "count_of_approved_trials"]

base_features = ["label","smi", "count_common_targets", "count_drug_targets", "count_of_approved_trials", 'gastiger_charges', 'tpsa']
#base_features = ["label", "smi", "count_common_targets", "count_drug_targets", "atc"]

experiments = [
#     {'data' : 'main', 'fp' : smi2vec, 'additional_features' : [], 'n_clusters' : 8},
#     {'data' : 'ki', 'fp' : smi2vec, 'additional_features' : [], 'n_clusters' : 6},
#     {'data' : 'ki', 'fp' : smi2vec, 'additional_features' : ['minki'], 'n_clusters' : 6},
#     {'data' : 'ki', 'fp' : smi2vec, 'additional_features' : ['minki', 'maxki', 'medianki'], 'n_clusters' : 6},
#     {'data' : 'stdval', 'fp' : smi2vec, 'additional_features' : [], 'n_clusters' : 6},
#     {'data' : 'stdval', 'fp' : smi2vec, 'additional_features' : ['medianstdval'], 'n_clusters' : 6},
#     {'data' : 'stdval', 'fp' : smi2vec, 'additional_features' : ['medianstdval', 'maxstdval', 'minsdtval'], 'n_clusters' : 6},

#     {'data' : 'main', 'fp' : maccs, 'additional_features' : [], 'n_clusters' : 8},
#     {'data' : 'ki', 'fp' : maccs, 'additional_features' : [], 'n_clusters' : 6},
#     {'data' : 'ki', 'fp' : maccs, 'additional_features' : ['minki'], 'n_clusters' : 6},
#     {'data' : 'ki', 'fp' : maccs, 'additional_features' : ['minki', 'maxki', 'medianki'], 'n_clusters' : 6},
#     {'data' : 'stdval', 'fp' : maccs, 'additional_features' : [], 'n_clusters' : 6},
#     {'data' : 'stdval', 'fp' : maccs, 'additional_features' : ['medianstdval'], 'n_clusters' : 6},
#     {'data' : 'stdval', 'fp' : maccs, 'additional_features' : ['medianstdval', 'maxstdval', 'minsdtval'], 'n_clusters' : 6},

#     {'data' : 'main_v1', 'fp' : smi2vec, 'additional_features' : [], 'n_clusters' : 8},
#     {'data' : 'ki_v1', 'fp' : smi2vec, 'additional_features' : [], 'n_clusters' : 6},
#     {'data' : 'ki_v1', 'fp' : smi2vec, 'additional_features' : ['minki'], 'n_clusters' : 6},
#     {'data' : 'ki_v1', 'fp' : smi2vec, 'additional_features' : ['minki', 'maxki', 'medianki'], 'n_clusters' : 6},
#     {'data' : 'stdval_v1', 'fp' : smi2vec, 'additional_features' : [], 'n_clusters' : 6},
#     {'data' : 'stdval_v1', 'fp' : smi2vec, 'additional_features' : ['medianstdval'], 'n_clusters' : 6},
#     {'data' : 'stdval_v1', 'fp' : smi2vec, 'additional_features' : ['medianstdval', 'maxstdval', 'minsdtval'], 'n_clusters' : 6},
    
#     {'data' : 'main_v1', 'fp' : maccs, 'additional_features' : [], 'n_clusters' : 8},
#     {'data' : 'ki_v1', 'fp' : maccs, 'additional_features' : [], 'n_clusters' : 6},
#     {'data' : 'ki_v1', 'fp' : maccs, 'additional_features' : ['minki'], 'n_clusters' : 6},
#     {'data' : 'ki_v1', 'fp' : maccs, 'additional_features' : ['minki', 'maxki', 'medianki'], 'n_clusters' : 6},
#     {'data' : 'stdval_v1', 'fp' : maccs, 'additional_features' : [], 'n_clusters' : 6},
#     {'data' : 'stdval_v1', 'fp' : maccs, 'additional_features' : ['medianstdval'], 'n_clusters' : 6},
#     {'data' : 'stdval_v1', 'fp' : maccs, 'additional_features' : ['medianstdval', 'maxstdval', 'minsdtval'], 'n_clusters' : 6},
    
#     {'data' : 'main', 'fp' : mol2vec, 'additional_features' : [], 'n_clusters' : 8},
#     {'data' : 'ki', 'fp' : mol2vec, 'additional_features' : [], 'n_clusters' : 6},
#     {'data' : 'ki', 'fp' : mol2vec, 'additional_features' : ['minki'], 'n_clusters' : 6},
#     {'data' : 'ki', 'fp' : mol2vec, 'additional_features' : ['minki', 'maxki', 'medianki'], 'n_clusters' : 6},
#     {'data' : 'stdval', 'fp' : mol2vec, 'additional_features' : [], 'n_clusters' : 6},
#     {'data' : 'stdval', 'fp' : mol2vec, 'additional_features' : ['medianstdval'], 'n_clusters' : 6},
#     {'data' : 'stdval', 'fp' : mol2vec, 'additional_features' : ['medianstdval', 'maxstdval', 'minsdtval'], 'n_clusters' : 6},
    
    
#     {'data' : 'main_v1', 'fp' : mol2vec, 'additional_features' : [], 'n_clusters' : 8},
#     {'data' : 'ki_v1', 'fp' : mol2vec, 'additional_features' : [], 'n_clusters' : 6},
#     {'data' : 'ki_v1', 'fp' : mol2vec, 'additional_features' : ['minki'], 'n_clusters' : 6},
    {'data' : 'ki_v1', 'fp' : mol2vec, 'additional_features' : ['minki', 'maxki', 'medianki'], 'n_clusters' : 6},
    {'data' : 'stdval_v1', 'fp' : mol2vec, 'additional_features' : [], 'n_clusters' : 6},
    {'data' : 'stdval_v1', 'fp' : mol2vec, 'additional_features' : ['medianstdval'], 'n_clusters' : 6},
    {'data' : 'stdval_v1', 'fp' : mol2vec, 'additional_features' : ['medianstdval', 'maxstdval', 'minsdtval'], 'n_clusters' : 6},
    
    {'data' : 'main', 'fp' : mol2vec_ft, 'additional_features' : [], 'n_clusters' : 8},
    {'data' : 'ki', 'fp' : mol2vec_ft, 'additional_features' : [], 'n_clusters' : 6},
    {'data' : 'ki', 'fp' : mol2vec_ft, 'additional_features' : ['minki'], 'n_clusters' : 6},
    {'data' : 'ki', 'fp' : mol2vec_ft, 'additional_features' : ['minki', 'maxki', 'medianki'], 'n_clusters' : 6},
    {'data' : 'stdval', 'fp' : mol2vec_ft, 'additional_features' : [], 'n_clusters' : 6},
    {'data' : 'stdval', 'fp' : mol2vec_ft, 'additional_features' : ['medianstdval'], 'n_clusters' : 6},
    {'data' : 'stdval', 'fp' : mol2vec_ft, 'additional_features' : ['medianstdval', 'maxstdval', 'minsdtval'], 'n_clusters' : 6},
    
    {'data' : 'main_v1', 'fp' : mol2vec_ft, 'additional_features' : [], 'n_clusters' : 8},
    {'data' : 'ki_v1', 'fp' : mol2vec_ft, 'additional_features' : [], 'n_clusters' : 6},
    {'data' : 'ki_v1', 'fp' : mol2vec_ft, 'additional_features' : ['minki'], 'n_clusters' : 6},
    {'data' : 'ki_v1', 'fp' : mol2vec_ft, 'additional_features' : ['minki', 'maxki', 'medianki'], 'n_clusters' : 6},
    {'data' : 'stdval_v1', 'fp' : mol2vec_ft, 'additional_features' : [], 'n_clusters' : 6},
    {'data' : 'stdval_v1', 'fp' : mol2vec_ft, 'additional_features' : ['medianstdval'], 'n_clusters' : 6},
    {'data' : 'stdval_v1', 'fp' : mol2vec_ft, 'additional_features' : ['medianstdval', 'maxstdval', 'minsdtval'], 'n_clusters' : 6},
    
    {'data' : 'main', 'fp' : morgan, 'additional_features' : [], 'n_clusters' : 8},
    {'data' : 'ki', 'fp' : morgan, 'additional_features' : [], 'n_clusters' : 6},
    {'data' : 'ki', 'fp' : morgan, 'additional_features' : ['minki'], 'n_clusters' : 6},
    {'data' : 'ki', 'fp' : morgan, 'additional_features' : ['minki', 'maxki', 'medianki'], 'n_clusters' : 6},
    {'data' : 'stdval', 'fp' : morgan, 'additional_features' : [], 'n_clusters' : 6},
    {'data' : 'stdval', 'fp' : morgan, 'additional_features' : ['medianstdval'], 'n_clusters' : 6},
    {'data' : 'stdval', 'fp' : morgan, 'additional_features' : ['medianstdval', 'maxstdval', 'minsdtval'], 'n_clusters' : 6},
    
    {'data' : 'main_v1', 'fp' : morgan, 'additional_features' : [], 'n_clusters' : 8},
    {'data' : 'ki_v1', 'fp' : morgan, 'additional_features' : [], 'n_clusters' : 6},
    {'data' : 'ki_v1', 'fp' : morgan, 'additional_features' : ['minki'], 'n_clusters' : 6},
    {'data' : 'ki_v1', 'fp' : morgan, 'additional_features' : ['minki', 'maxki', 'medianki'], 'n_clusters' : 6},
    {'data' : 'stdval_v1', 'fp' : morgan, 'additional_features' : [], 'n_clusters' : 6},
    {'data' : 'stdval_v1', 'fp' : morgan, 'additional_features' : ['medianstdval'], 'n_clusters' : 6},
    {'data' : 'stdval_v1', 'fp' : morgan, 'additional_features' : ['medianstdval', 'maxstdval', 'minsdtval'], 'n_clusters' : 6},
]

df = read_sheets(sheets['ki'])
X_m2v = maccs(df)
X_m2v.shape
kmeans = KMeans(n_clusters=6, random_state = 10).fit(X_m2v)
sn.histplot(kmeans.labels_)

In [ ]:
#CSV to store the final results
final_output_csv = None

# classifiers used for experiment.
names = ["RF", "ExtRa Trees",  "Adaboost","SVM", "Log. Reg.", "kNN" , "Naive Bayes", "NN"]
def kmeans_based_split(df, n_clusters):
    """
    Splits the df into n subsets based on KMeans clustering of MACCS fingerprints.
    Returns a list of dataframes
    """
    #print(len(df))
    X_maccs = maccs(df.copy())
    kmeans = KMeans(n_clusters=n_clusters).fit(X_maccs)
    labels = kmeans.labels_
    assert labels.shape[0] == len(df)
    dataframes = []
    for i in range(n_clusters):
        df_subset = df.loc[labels == i].copy()
        dataframes.append(df_subset)
    #print([len(i) for i in dataframes], sum([len(i) for i in dataframes]))
    assert sum([len(i) for i in dataframes]) == len(df)
    return dataframes
    
for experiment in experiments: 
    print(experiment)
    
    #output csv for this experiment
    output_csv = {'intervention':[], 'train_size' : [], 'test_size' : [], 'confusion' : [],\
                  'auc' : []}
    for i in "aprf":
        for j in names:
            output_csv[i+"_"+j] = []
            if i != 'a':
                output_csv[i+"0_"+j] = []
            
            
    print("Experiment:", experiment)
    df = read_sheets(sheets[experiment['data']])
    # Drop nan values from required columns
    df = df.loc[:, df.columns.isin(base_features+ experiment['additional_features'] + ["minsdtval", "minki", "drugbank id", "count_common_targets"])].dropna()
    df = df.loc[:, df.columns.isin(base_features+ experiment['additional_features'] + ["drugbank id", "count_common_targets"])].dropna()
    df.reset_index(drop=True, inplace=True)
    df = df[df['count_common_targets'] <=6]
    experiment['n_passes'] = len(df[df['label'] == 0])
    experiment['n_fails'] = len(df[df['label'] == 1])
    
    splits =  kmeans_based_split(df, experiment['n_clusters'])
    df = None
    y = [df.loc[:, df.columns.isin(['label'])] for df in splits]
    y = [df.iloc[:, 0].to_numpy() for df in y]
    sample = splits[0].loc[:, splits[0].columns.isin(base_features [2:] + experiment['additional_features'] + ["drugbank id"])]
    print(sample.columns)
    X = [df.loc[:, df.columns.isin(base_features [2:] + experiment['additional_features'] + ["drugbank id"])].to_numpy() for df in splits]
    assert X[0].shape[1] == len(base_features[2:] + experiment['additional_features']+ ['drugbank id']), "SOME FEATURES MISSING!!!!"
    assert y[0].shape[0] == X[0].shape[0], "missing rows!"

    for i in range(experiment['n_clusters']):
        X[i] = np.concatenate([X[i], experiment['fp'](splits[i])], 1)
    
    # obtain the best hyperparameters for current setup via grid search.
    classifiers1 = get_hyperparams(np.concatenate(X, axis = 0)[:,1:], np.concatenate(y, axis = 0))
    print(classifiers1)
    kfolds = [StratifiedKFold(n_splits = 5).split(X[i], y[i]) for i in range(experiment['n_clusters'])]
    
    for split in tqdm(range(5)):
        gc.collect()
        X_trains, X_tests, y_trains, y_tests = [], [], [], []
        
        for i in range(experiment['n_clusters']):
            train_idx, test_idx = next(kfolds[i])
            X_trains.append(X[i][train_idx])
            X_tests.append(X[i][test_idx])
            y_trains.append(y[i][train_idx])
            y_tests.append(y[i][test_idx])
            
        
        X_train = np.concatenate(X_trains, 0)
        X_test = np.concatenate(X_tests, 0)
        y_train = np.concatenate(y_trains, 0)
        y_test = np.concatenate(y_tests, 0)
        
        train_drugs = X_train[:,0].tolist()
        test_mask = []
        for i in range(X_test.shape[0]):
            test_mask.append(X_test[i][0] not in train_drugs)
        X_train = X_train[:,1:]
        
        test_mask = np.asarray(test_mask, dtype = bool)
        X_test = X_test[test_mask]
        y_test = y_test[test_mask]
        cnt_mask = np.where(X_test[:, 1] == 1)
        X_test = X_test[cnt_mask]
        y_test = y_test[cnt_mask]
        X_test = X_test[:,1:]
        print(X_train.shape, X_test.shape)
        
        classifiers = get_classifiers(classifiers1)
        
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        
        accuracy_values = []
        pvalues = []
        rvalues = []
        fvalues = []
        
        pvalues0 = []
        rvalues0 = []
        fvalues0 = []
        
        nmes = []
        logreg_confusion = None
        logreg_auc = None
        
        for idx, clf in enumerate(classifiers):
            clf.fit(X_train, y_train)
            pred = clf.predict(X_test)
            accuracy_values.append(
                accuracy_score(y_test, pred)
            )
            p, r, f, _ = precision_recall_fscore_support(y_test, pred, pos_label = 0, average = "binary")
            pvalues.append(p)
            rvalues.append(r)
            fvalues.append(f)
            p0, r0, f0, _ = precision_recall_fscore_support(y_test, pred, pos_label = 1, average = "binary")
            pvalues0.append(p0)
            rvalues0.append(r0)
            fvalues0.append(f0)
            nmes.append(names[idx])
            
            if names[idx] == "Log. Reg.":
                logreg_confusion = confusion_matrix(y_test, pred).__str__()
                pred = clf.predict_proba(X_test)
                try:
                    logreg_auc = roc_auc_score(y_test, pred[:,0])
                except ValueError:
                    logreg_auc = -1
        
        acc_dict = {
            "a" : accuracy_values,
            "p" : pvalues, 
            "r" : rvalues, 
            "f" : fvalues,
            "p0" : pvalues0, 
            "r0" : rvalues0, 
            "f0" : fvalues0,
            "algorithm" : nmes
        }
        
        output_csv["confusion"].append(logreg_confusion)
        output_csv["auc"].append(logreg_auc)
        output_csv["intervention"].append(split)
        output_csv['train_size'].append(X_train.shape[0])
        output_csv['test_size'].append(X_test.shape[0])
        for ii in names:
            for jj in "arpf":
                output_csv[jj+"_"+ii].append(acc_dict[jj][acc_dict["algorithm"].index(ii)])
                if jj != 'a':
                    output_csv[jj+"0_"+ii].append(acc_dict[jj+'0'][acc_dict["algorithm"].index(ii)])
        
    output_csv['intervention'].append("mean")
    output_csv['confusion'].append("mean")

    for i in output_csv:
        if i not in ["intervention", "confusion"]:
            output_csv[i].append(sum(output_csv[i])/len(output_csv[i]))

    output_csv['intervention'].append("std")
    output_csv['confusion'].append("std")
    for i in output_csv:
        if i not in ["intervention", "confusion"]:
            output_csv[i].append(np.std(np.array(output_csv[i][:-1])))
    for i in output_csv:
        if i != "intervention":
            output_csv[i].extend([np.nan]*15)
        else:
            output_csv[i].extend(["data : %s, fp : %s, additional_features : %s, passes, fails : %d, %d"%(experiment['data'], \
                            experiment['fp'].__name__, ",".join(experiment['additional_features'])
                            , experiment['n_passes'], experiment['n_fails'])] + [np.nan]*14)

    if final_output_csv is None:
        final_output_csv = deepcopy(output_csv)
    else:
        for i in output_csv:
            final_output_csv[i].extend(output_csv[i])
    df = pd.DataFrame(final_output_csv)
    df.to_csv("cluster_experiments_cnt_one.csv")
